In [ ]:
import random as rand
import nltk
import lightrdf
import re
from nltk.corpus import wordnet as wn

nltk.download('wordnet')
nltk.download('omw-1.4')

list_of_triplets = list()


def rdf_parsing():
    document = lightrdf.RDFDocument("food.rdf")
    for triple in document.search_triples(None, None, None):
        print(triple)
    return document


def composed_rdf_parsing():
    document = lightrdf.RDFDocument("ontology/food.rdf")
    lines = list()
    for triple in document.search_triples(None, None, None):
        lines.append(triple)
    for ind, line in enumerate(lines):
        if str(line[2]).startswith('_:riog') and not str(line[0]).startswith('_:riog'):
            count = ind
            while count < len(lines):
                if str(lines[count][2]).split("#")[-1][:-1].startswith('has'):
                    if count + 1 < len(lines):
                        if not str(lines[count + 1][2]).split("#")[-1][:-1].startswith('Restriction'):
                            concept_nr1 = line[0].split("#")[-1][:-1]
                            rel = lines[count][2].split("#")[-1][:-1]
                            concept_nr2 = lines[count + 1][2].split("#")[-1][:-1]
                            print(concept_nr1, ' ', rel, ' ', concept_nr2)
                count = count + 1


def show_triplets_names():
    global list_of_triplets
    document = rdf_parsing()
    for triple in document.search_triples(None, None, None):
        # verificam daca fiecare elment are un nume (are un # in el)
        if "#" in triple[0] and "#" in triple[1] and "#" in triple[2]:
            # luam numele fiecarui element
            triple = (triple[0].split("#")[-1][:-1], triple[1].split("#")[-1][:-1], triple[2].split("#")[-1][:-1])
            # spatii intre cuvinte
            triple = (re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', triple[0]),
                      re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', triple[1]),
                      re.sub(r'((?<=[a-z])[A-Z]|(?<!\A)[A-Z](?=[a-z]))', r' \1', triple[2]))
            list_of_triplets.append(triple)
    for triple in list_of_triplets:
        print(triple)


def questionnaire():
    global list_of_triplets
    question_counter = 0
    while True:
        question_counter += 1
        print(question_counter)
        triple = list_of_triplets[question_counter]
        print("Question " + str(question_counter) + ":")
        # stergem elementul din triplet
        hid_elm = rand.randrange(2)
        if hid_elm == 0:
            q = "what's a " + triple[1] + " of " + triple[2] + "?"
        elif hid_elm == 1:
            q = "what's the relationship between " + triple[0] + " and " + triple[2] + "?"
        else:
            q = "what's the " + triple[1] + " of " + triple[0] + "?"
        print(q)
        a = input("answer: ")
        if a.lower() == triple[hid_elm].lower():
            print("correct!")
        else:
            print("wrong! the correct answer is: " + triple[hid_elm])
        print("conitnue? (y/n)")
        a = input("answer: ")
        if a == "n":
            break
            
def extended_questionnaire():
    global list_of_triplets
    question_counter = 0
    while True:
        question_counter += 1
        print(question_counter)
        triple = list_of_triplets[question_counter]
        print("question " + str(question_counter) + ":")
        # stergem elementul din triplet
        hid_elm = rand.randrange(2)
        if hid_elm == 0:
            q = "what's a " + rand.choice(get_all_possibilities(triple[1])) + \
            " of " +  rand.choice(get_all_possibilities(triple[2])) + "?"
        elif hid_elm == 1:
            q = "what's the relationship between " + rand.choice(get_all_possibilities(triple[0])) + \
            " and " + rand.choice(get_all_possibilities(triple[2])) + "?"
        else:
            q = "what's the " + rand.choice(get_all_possibilities(triple[1])) + " of " + \
            rand.choice(get_all_possibilities(triple[0])) + "?"
        print(q)
        a = input("answer: ")
        if a in get_hypernyms(a):
            print("it's a hypernym")
        elif a in get_meronyms(a):
            print("it's a meronym")
        elif a in get_synonyms(a):
            print("it's a synonym")
       
        if a.lower() in get_all_possibilities(triple[hid_elm].lower()):
            print("correct!")
        else:
            print("wrong! the correct answer is: " + triple[hid_elm])
        print("conitnue? (y/n)")
        a = input("answer: ")
        if a == "n":
            break



# we need only the first sense of the word - to modify
def get_synonyms(w):
    synonyms = list()
    try: 
        synset = wn.synsets(w)[0]
        for elm in synset.lemmas():
            synonyms.append(elm.name())
    except:
        synonyms.append(w)
    return synonyms


def get_hypernyms(word):
    hypernyms = []
    try: 
        syn = wn.synsets(word)[0]
        for elm in syn.hypernyms():
            hypernyms.append(elm.name())
    except:
        pass
    return hypernyms


def get_meronyms(word):
    meronyms = []
    try: 
        syn = wn.synsets(word)[0]
        for elm in syn.part_meronyms():
            meronyms.append(elm.name())
    except:
        pass
    return meronyms


def get_synsets(word):
    synonyms = []
    for syn in wn.synsets(word):
        print(syn, " has the following synonyms: ")
        for l in syn.lemmas():
            synonyms.append(l.name())
            print("->", l.name())
    return synonyms
        

def get_all_possibilities(word):
    possible_answers = []
    possible_answers.extend(get_synonyms(word))
    possible_answers.extend(get_meronyms(word))
    possible_answers.extend(get_hypernyms(word))
    possible_answers = list(dict.fromkeys(possible_answers)) # eliminate all duplicates
    return possible_answers


def main():
    get_synsets(input())
    show_triplets_names()
    extended_questionnaire()


if __name__ == '__main__':
    main()


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/volentiralexandra/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/volentiralexandra/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


drink
Synset('drink.n.01')  has the following synonyms: 
-> drink
Synset('drink.n.02')  has the following synonyms: 
-> drink
-> drinking
-> boozing
-> drunkenness
-> crapulence
Synset('beverage.n.01')  has the following synonyms: 
-> beverage
-> drink
-> drinkable
-> potable
Synset('drink.n.04')  has the following synonyms: 
-> drink
Synset('swallow.n.02')  has the following synonyms: 
-> swallow
-> drink
-> deglutition
Synset('drink.v.01')  has the following synonyms: 
-> drink
-> imbibe
Synset('drink.v.02')  has the following synonyms: 
-> drink
-> booze
-> fuddle
Synset('toast.v.02')  has the following synonyms: 
-> toast
-> drink
-> pledge
-> salute
-> wassail
Synset('drink_in.v.01')  has the following synonyms: 
-> drink_in
-> drink
Synset('drink.v.05')  has the following synonyms: 
-> drink
-> tope
('<http://www.w3.org/TR/2003/PR-owl-guide-20031209/food>', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://www.w3.org/2002/07/owl#Ontology>')
('<http://www.w3.org/TR/200

answer: type
it's a synonym
correct!
conitnue? (y/n)
